In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!unzip /content/movie_data.zip

Archive:  /content/movie_data.zip
replace movie_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: movie_data.csv          


In [ ]:
df = pd.DataFrame()
df = pd.read_csv('/content/movie_data.csv')
df.head(20)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
5,"I saw this film on September 1st, 2005 in Indi...",1
6,"Maybe I'm reading into this too much, but I wo...",1
7,I felt this film did have many good qualities....,1
8,This movie is amazing because the fact that th...,1
9,"""Quitting"" may be as much about exiting a pre-...",1


In [ ]:
df.shape

(50000, 2)

In [ ]:
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
import nltk
nltk.download('punkt') # Used for sentence tokenizer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

review_lines = list()
lines = df['review'].values.tolist()
print(len(lines))
print(lines[:5])


50000
["I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.", 'Actor turned director Bill Paxton follows up his promising debut, the Gothic-horror "Frailty", with this family friendly sports drama about the 1913 U.S. Open where a young American caddy rises 

In [ ]:
for line in lines:
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]

    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    #Replace the characters in the first argument with the corresponding characters in the second argument
    #Third argument represents a string of characters that must be removed from the original string
    stripped = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()] #.isalpha returns True if the chaarcters are aphabets

    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [ ]:
import gensim # Process plain text

EMBEDDING_DIM = 100
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, min_count=1, sg=0)
'''
size : Number of dimensions of the embeddings
window : sliding window size
min_count : Minimum count of words to consider while training a model. Default is 5
sg : if you want to use cbow(when 0) or sg(when 1)
'''
# vocab size
words = list(model.wv.vocab) # wv is the object that contain mappings between words and embeddings
print('Vocabulary size: %d' % len(words))

Vocabulary size: 133264


In [ ]:
model['man']
len(model['man'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


100

In [ ]:
# save model in ASCII (word2vec) format
filename = 'imdb_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False) # Don't want to store it in binary format. Default : binary format to save space

In [ ]:
# let us try some utility functions of gensim word2vec more details here

model.wv.most_similar('money')#, topn =1)

[('cash', 0.6613119840621948),
 ('deductments', 0.6331372261047363),
 ('cost', 0.6210497617721558),
 ('ticket', 0.6172667145729065),
 ('bucks', 0.6131108403205872),
 ('bobbitt', 0.6072616577148438),
 ('bills', 0.6017158627510071),
 ('paid', 0.5984830260276794),
 ('desposal', 0.5893391370773315),
 ('dough', 0.5821436643600464)]

In [ ]:
#Let’s see the result of semantically reasonable word vectors (king - man + woman)
model.wv.most_similar(positive=['queen', 'boy'], negative=['girl'], topn=1)

[('princess', 0.7310706377029419)]

In [ ]:
#odd word out
print(model.wv.doesnt_match("woman king queen movie".split()))

movie


/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [ ]:
model.wv.similar_by_word("cat")

[('dog', 0.7910478115081787),
 ('mouse', 0.7626219391822815),
 ('monkey', 0.7270614504814148),
 ('bugs', 0.7208654284477234),
 ('dude', 0.7198402881622314),
 ('hat', 0.7099525928497314),
 ('pet', 0.6968947649002075),
 ('snake', 0.6838912963867188),
 ('daffy', 0.6773809194564819),
 ('bird', 0.6726865768432617)]

In [ ]:
print(model.similarity('boy', 'girl'))

0.8495133


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
import os

embeddings_index = {}
#f = open(os.path.join('', 'imdb_embedding_word2vec.txt'),  encoding = "utf-8")
f = open('/content/imdb_embedding_word2vec.txt')
for line in f:
  print(line)
  values = line.split()
  #print(values)
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs

f.close()


In [ ]:
len(embeddings_index)

133265

In [ ]:
max_length = 100 # try other options like mean of sentence lengths

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.2

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines) # Transforms text into a sequence of integers


In [ ]:
sequences

In [ ]:
# pad sequences
'''
pad_sequences : convert a list of sequences into a 2D numpy array of shape (num_samples,num_timesteps)
num_timesteps : maxlen(if provided) or length of the longest sequence
pre padding or truncating from the beginning is the default, can change it to post
'''
word_index = tokenizer_obj.word_index # Maps words to their numeric representation
print('Found %s unique tokens.' % len(word_index))
vocab_size=len(word_index)+1
print('The vocabulary size is: ', vocab_size)
review_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
sentiment =  df['sentiment'].values
print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', sentiment.shape)



In [ ]:
# split the data into a training set and a validation set
indices = np.arange(review_pad.shape[0])
#print(indices)
#print(len(indices))
np.random.shuffle(indices)
print(indices)
#print(len(indices))


[15737 29712 97733 ... 93545 19407 96488]


In [ ]:
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

In [ ]:
X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [ ]:
print('Shape of X_train_pad tensor:', X_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)

print('Shape of X_test_pad tensor:', X_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of X_train_pad tensor: (40000, 100)
Shape of y_train tensor: (40000,)
Shape of X_test_pad tensor: (10000, 100)
Shape of y_test tensor: (10000,)


In [ ]:
EMBEDDING_DIM =100
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word) #model[word]
  if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
    embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.initializers import Constant

# define model
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)
model.add(embedding_layer)
model.add(LSTM(units=32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Summary of the built model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          13326500  
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 13,343,557
Trainable params: 17,057
Non-trainable params: 13,326,500
_________________________________________________________________
None


In [ ]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=128, epochs=50, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Epoch 1/50
313/313 - 118s - loss: 0.5168 - accuracy: 0.7525 - val_loss: 0.4282 - val_accuracy: 0.8162 - 118s/epoch - 376ms/step
Epoch 2/50
313/313 - 108s - loss: 0.4007 - accuracy: 0.8260 - val_loss: 0.3705 - val_accuracy: 0.8430 - 108s/epoch - 346ms/step
Epoch 3/50
313/313 - 109s - loss: 0.3580 - accuracy: 0.8447 - val_loss: 0.3351 - val_accuracy: 0.8549 - 109s/epoch - 349ms/step
Epoch 4/50
313/313 - 108s - loss: 0.3383 - accuracy: 0.8544 - val_loss: 0.3234 - val_accuracy: 0.8633 - 108s/epoch - 345ms/step
Epoch 5/50
313/313 - 107s - loss: 0.3223 - accuracy: 0.8637 - val_loss: 0.3144 - val_accuracy: 0.8668 - 107s/epoch - 342ms/step
Epoch 6/50
313/313 - 108s - loss: 0.3144 - accuracy: 0.8655 - val_loss: 0.3121 - val_accuracy: 0.8669 - 108s/epoch - 345ms/step
Epoch 7/50
313/313 - 107s - loss: 0.3046 - accuracy: 0.8707 - val_loss: 0.3063 - val_accuracy: 0.8693 - 107s/epoch - 343ms/step
Epoch 8/50
313/313 - 109s - loss: 0.3038 - accuracy: 0.8693 - val_loss: 0.3000 - val_accuracy: 